# Youtube

## Functional

- Users can upload videos.
- Users can share and view videos.
- Users can search videos.
- Tracks likes/dislikes, number of views.
- Users can add and view comments.

## Non-functional

- High availability. 
- High reliability.
- Consistency can take a hit.
- Users should not feel lag watching videos.

## Capacity

### Assume
- 1.5B total users.
- 800M daily active users.
- Users watch 5 videos per day.
- Ratio of upload:view is 1:200. 
- 500 hours of videos are uploaded per minute.
- 1 minute vidoe takes 50MB storage.
- 1 minute vidoe takes 10MB bandwidth.

### Traffic
- Video views per second
    - 800M * 5 / 86400 = 46K videos per second.
- Video uploads per second
    - 46K / 200 = 230 videos per second.

### Storage
- 500 * 60 * 50 = 1500GB per minute.
    
### Bandwidth
- Incoming: 500 * 60 * 10 = 300GB per minute.
- Outgoing: 300GB * 200 = 60TB per minute.

## API

- uploadVideo(api_dev_key, video_title, video_description, tags[], category_id, default_language, recording_details, video_contents)
    - Return 202 on success.
- searchVideo(api_dev_key, search_query, user_location, max_videos_to_return)
    - Return JSON with the list of videos.
- streamVideo(api_dev_key, video_id, offset, codec, resolution)
    - Return stream.

## Design

<img src="img/youtube1.png" style="width:600px;height:400px;"> 

- Processing queue: each uploading video shall enter this queue first. 
- Encoder: encode each uploaded vidoe into different formats.
- Thumbnails generator: generate thumbnaul for each video.
- Video and Thumbnail storage: store video and thumbnail.
- User Database: store user information.
- Video metadata storage: store video metadata.

<img src="img/youtube2.png" style="width:800px;height:600px;">

### Where to store videos
- HDFS

### How to manage read traffic
- Segregate read traffic and write traffic.
- Distribute read traffic to different servers.
- For video metadata, write to primary and read from secondary.

### Where to store thumbnail
- Bigtable
    - Combines multiple files into one block to store.
    - Very efficient reading small amount of data.

### Metadata sharding
- Store based on UserID? It is hard to maintain uniform distribution because some users upload more than others.
- Store based on VideoID? Solves above problem. Can cache hot vidoes in front of DB servers.

### Deduplication
- When users upload vidoes, run video matching algorithm.

## DB
- Store video metadata in relational DB.

### Video
- VideoID
- Title
- Description
- Size
- Thumbnail
- Uploader
- Total number of like
- Total number of dislikes
- Total number of views

### VideoComment
- CommentID
- VideoID
- UserID
- Comment
- TimeOfCreation

### User
- UserID
- Name
- Email

## Load-balancing
- Consistent hashing between cache servers.

## Caching
- Store hot vidoes in cache servers.
- Use LRU eviction rule.